# Advent of Code 2022

## Day 17: Pyroclastic Flow

Solution code by [leechristie](https://github.com/leechristie) for Advent of Code 2022.

### Imports

In [390]:
from enum import Enum
from typing import Iterator, Optional
from collections import namedtuple
import tqdm.notebook as tqdm

### Directions

In [391]:
class Direction(Enum):
    LEFT = 1
    RIGHT = 2
    UP = 3
    DOWN = 4

    def __str__(self):
        assert self in {Direction.LEFT, Direction.RIGHT, Direction.UP, Direction.DOWN}
        if self == Direction.LEFT:
            return '←'
        if self == Direction.RIGHT:
            return '→'
        if self == Direction.UP:
            return '↑'
        return '↓'

    def velocity(self):
        assert self in {Direction.LEFT, Direction.RIGHT, Direction.UP, Direction.DOWN}
        if self == Direction.LEFT:
            return Point(x=-1, y=0)
        if self == Direction.RIGHT:
            return Point(x=1, y=0)
        if self == Direction.UP:
            return Point(x=0, y=1)
        return Point(x=0, y=-1)

    def __repr__(self):
        return str(self)

    @staticmethod
    def parse(symbol: str) -> 'Direction':
        assert symbol in {'<', '>'}
        if symbol == '<':
            return Direction.LEFT
        return Direction.RIGHT

INPUT_FILE = 'data/input17.txt'

with open(INPUT_FILE) as file:
    line = next(file).strip()
    DIRECTIONS = [Direction.parse(symbol) for symbol in line]

def infinite_direction_source(*, limit: Optional[int] = None) -> Iterator[Direction]:
    global DIRECTIONS
    assert limit is None or (type(limit) == int and limit >= 0)
    count = 0
    while True:
        for rv in DIRECTIONS:
            if limit is not None and count >= limit:
                return
            yield rv
            count += 1

### Rocks Shapes

In [392]:
class RockShape:

    __slots__ = ['slices']

    def __init__(self, encoding: list[str]):
        assert (len(encoding) >= 1), f'{len(encoding) = }'
        length = None
        slices = []
        for line in encoding:
            assert (len(line) >= 1), f'encoding has empty line'
            if length is None:
                length = len(line)
            else:
                assert (len(line) == length), f'encoding is not rectangular'
            for char in line:
                assert (char in {'.', '#'}), f'unknown character in encoding'
            slices.append(tuple([char == '#' for char in line]))
        self.slices = tuple(slices)

    def __str__(self):
        rv = ''
        for line in self.slices:
            for filled in line:
                if filled:
                    rv += '@'
                else:
                    rv += ' '
            rv += '\n'
        return rv

    def __repr__(self):
        return str(self)

ROCK_SHAPES = [
    RockShape(['####']),              # minus
    RockShape(['.#.', '###', '.#.']), # plus
    RockShape(['..#', '..#', '###']), # J
    RockShape(['#', '#', '#', '#']),  # I
    RockShape(['##', '##'])           # square
]

def infinite_rock_shape_source(*, limit: Optional[int] = None) -> Iterator[RockShape]:
    global ROCK_SHAPES
    assert limit is None or (type(limit) == int and limit >= 0)
    count = 0
    while True:
        for rv in ROCK_SHAPES:
            if limit is not None and count >= limit:
                return
            yield rv
            count += 1

### Falling Rocks

In [393]:
# noinspection PyTypeChecker
Point = namedtuple('Point', ['x', 'y'])

def add(a: Point, b: Point) -> Point:
    return Point(x=a.x+b.x, y=a.y+b.y)

def negate(a: Point) -> Point:
    return Point(x=-a.x, y=-a.y)

In [394]:
class FallingRock:

    __slots__ = ['shape', 'bottom_left']

    def __init__(self, shape: RockShape, bottom_left: Point):
        self.shape = shape
        self.bottom_left = bottom_left

    def to_points(self) -> set[Point]:
        rv = []
        current_y = self.bottom_left.y
        for neg_y_offset, cross_section in reversed(list(enumerate(self.shape.slices))):
            current_x = self.bottom_left.x
            for x_offset, is_rock in enumerate(cross_section):
                if is_rock:
                    rv.append(Point(x=current_x, y=current_y))
                current_x += 1
            current_y += 1
        return set(rv)

    def attempt_move(self, direction: Direction, chamber: 'Chamber'):

        velocity = direction.velocity()
        for p in self.to_points():
            assert (p not in chamber.points), f'ready to move rock, but {p} is landed'

        self.bottom_left = add(self.bottom_left, velocity)
        points = self.to_points()
        crashed = False

        # check for going outside area
        for p in points:
            if p.y < 0 or p.x < 0 or p.x >= Chamber.WIDTH:
                crashed = True
                break

        # check for landing
        if not crashed:
            for p in points:
                if p in chamber.points:
                    crashed = True
                    break

        # move back
        if crashed:
            self.bottom_left = add(self.bottom_left, negate(velocity))
            points = self.to_points()
            for p in points:
                assert (p not in chamber.points), f'moved rock back, but {p} is landed'
            return False

        return True

### Chamber

In [395]:
class Chamber:

    WIDTH = 7

    __slots__ = ['points', 'rocks', 'stack_height']

    def __init__(self):
        self.points: set[Point] = set()
        self.rocks: set[FallingRock] = set()
        self.stack_height = 0


    def land(self, falling_rock: FallingRock) -> None:
        for p in falling_rock.to_points():
            assert (p not in self.points), f'trying to land rock but aleady landed point {p}'
            self.points.add(p)
            self.rocks.add(falling_rock)
            if p.y + 1 > self.stack_height:
                self.stack_height = p.y + 1

    def spawn_rock(self, shape: RockShape) -> FallingRock:
        start = Point(x=2, y=3+self.stack_height)
        return FallingRock(shape, start)

    def render(self, falling_rock: Optional[FallingRock] = None) -> None:
        points = falling_rock.to_points() if falling_rock else set()
        start_height = self.stack_height
        if falling_rock is not None:
            new_top = falling_rock.bottom_left.y + len(falling_rock.shape.slices)
            if new_top > start_height:
                start_height = new_top
        for y in range(start_height, -1, -1):
            print('🔳', end='')
            for x in range(0, Chamber.WIDTH):
                p = Point(x=x, y=y)
                if p in self.points:
                    print('🟥', end='')
                elif p in points:
                    print('🟩', end='')
                else:
                    print('⬛', end='')
            print(f'🔳 {y}')
        print(f'🔳🔳🔳🔳🔳🔳🔳🔳🔳\tstack height = {self.stack_height}')
        print()

### Part 1

In [396]:
chamber = Chamber()

directions = infinite_direction_source()

verbose = False
print_each_block = False
print_end = False

num_blocks = 2022

for shape in tqdm.tqdm(infinite_rock_shape_source(limit=num_blocks), total=num_blocks):

    rock: Optional[FallingRock] = chamber.spawn_rock(shape)
    if verbose:
        print('rock begins falling')
        chamber.render(rock)

    while rock is not None:

        direction = next(directions)
        if verbose:
            print(f'jet of gas pushes {direction}', end= '')
        moved = rock.attempt_move(direction, chamber)
        if verbose:
            if not moved:
                print(' but nothing happened', end='')
            print(':')
            chamber.render(rock)

        direction = Direction.DOWN
        if verbose:
            print('rock falls 1 unit', end= '')
        moved = rock.attempt_move(direction, chamber)
        if not moved:
            if verbose:
                print(' causing it to come to a rest', end='')
            chamber.land(rock)
            rock = None
        if verbose:
            print(':')
        if verbose or print_each_block:
            chamber.render(rock)

print('Simulation ended.')
print(f'stack height is {chamber.stack_height}')
print()
if not verbose and not print_each_block:
    if print_end:
        chamber.render()

  0%|          | 0/2022 [00:00<?, ?it/s]

Simulation ended.
stack height is 3232

